In [43]:
import json
import itertools

def load_experiment_parameters(file_path="params_exp_short.json"):
    try:
        with open(file_path, "r") as f:
            params = json.load(f)
        return params["simulation_parameters"]
    except Exception as e:
        pass

# Sample parameter set for testing
params = load_experiment_parameters()
params1 = {
    "year": [1000],
    "num_house": [20, 80],
    "land_cells": [80],
    "prod_multiplier": [2],
    "fishing_discount": [1],
    "fallow_period": [2, 10],
    "spare_food_enabled": [True, False],
    "conditions": {
        "use_fertility": [True],
        "check_gender": [True],
        "check_land": [False, True]
    }
}

def full_factorial(params):
    param_keys = [k for k in params.keys() if k != "conditions"]
    condition_keys = list(params["conditions"].keys())
    
    param_values = [params[k] if isinstance(params[k], list) else [params[k]] for k in param_keys]
    condition_values = [params["conditions"][k] if isinstance(params["conditions"][k], list) else [params["conditions"][k]] for k in condition_keys]
    
    param_combinations = list(itertools.product(*param_values))
    condition_combinations = list(itertools.product(*condition_values))
    
    all_combinations = list(itertools.product(param_combinations, condition_combinations))
    return param_keys, condition_keys, all_combinations

def one_factor_at_a_time(params):
    import copy

    param_keys = [k for k in params if k != "conditions"]
    condition_keys = list(params.get("conditions", {}).keys())

    all_experiments = []

    # Get base (first) values
    base_param_values = [params[k][0] if isinstance(params[k], list) else params[k] for k in param_keys]
    base_condition_values = [params["conditions"][k][0] if isinstance(params["conditions"][k], list) else params["conditions"][k] for k in condition_keys]

    def replace_and_tuple(base_list, index, new_value):
        temp = copy.deepcopy(base_list)
        temp[index] = new_value
        return tuple(temp)

    # Vary one parameter at a time
    for i, key in enumerate(param_keys):
        values = params[key] if isinstance(params[key], list) else [params[key]]
        for v in values:
            if v == base_param_values[i]:
                continue  # skip base value (avoid duplicates)
            param_tuple = replace_and_tuple(base_param_values, i, v)
            condition_tuple = tuple(base_condition_values)
            all_experiments.append((param_tuple, condition_tuple))

    # Vary one condition at a time
    for i, key in enumerate(condition_keys):
        values = params["conditions"][key] if isinstance(params["conditions"][key], list) else [params["conditions"][key]]
        for v in values:
            if v == base_condition_values[i]:
                continue  # skip base value
            param_tuple = tuple(base_param_values)
            condition_tuple = replace_and_tuple(base_condition_values, i, v)
            all_experiments.append((param_tuple, condition_tuple))

    # Add base case as the first experiment
    all_experiments.insert(0, (tuple(base_param_values), tuple(base_condition_values)))

    return param_keys, condition_keys, all_experiments
# Run and display both
full_keys, full_conditions, full_combos = full_factorial(params)
ofat_keys, ofat_conditions, ofat_combos = one_factor_at_a_time(params)

print("Full factorial combinations ({} total):".format(len(full_combos)))

print("\nOne-factor-at-a-time combinations ({} total):".format(len(ofat_combos)))
for combo in ofat_combos:
    print(combo)


Full factorial combinations (131072 total):

One-factor-at-a-time combinations (18 total):
((1000, 20, 80, 2, 1, 0.05, 2, 1, 10, 60, 0.1, 2, 0.01, 10, 1, 5, 10, 0.2, 1.5, 2, 5, 1.5, 20, 1, 0.01, 2, 5, 20, 'simulation_plots.svg', 'numerical_output.csv', 'demog_vectors_scaled.csv', 'simulation_gif.gif', 0.1, 5, True, True, True, True), (True, True, True, False, True))
((1000, 80, 80, 2, 1, 0.05, 2, 1, 10, 60, 0.1, 2, 0.01, 10, 1, 5, 10, 0.2, 1.5, 2, 5, 1.5, 20, 1, 0.01, 2, 5, 20, 'simulation_plots.svg', 'numerical_output.csv', 'demog_vectors_scaled.csv', 'simulation_gif.gif', 0.1, 5, True, True, True, True), (True, True, True, False, True))
((1000, 20, 80, 2, 1, 0.5, 2, 1, 10, 60, 0.1, 2, 0.01, 10, 1, 5, 10, 0.2, 1.5, 2, 5, 1.5, 20, 1, 0.01, 2, 5, 20, 'simulation_plots.svg', 'numerical_output.csv', 'demog_vectors_scaled.csv', 'simulation_gif.gif', 0.1, 5, True, True, True, True), (True, True, True, False, True))
((1000, 20, 80, 2, 1, 0.05, 10, 1, 10, 60, 0.1, 2, 0.01, 10, 1, 5, 10, 0.2, 

In [39]:
type(params['conditions'])

dict

In [40]:
type(params1['conditions'])

dict